# Description: Sentiment Analysis of Covid19 Twitter dataset 

1. Preprocessing data
2. Creating labels using Vader Sentiment Analysis
3. Add modifications to labels according to context 
4. Saving results as csv and excel files

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


In [2]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_042fd5e199e2413b901031aefb0328bd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='jIT-NuySrFbhm93nCeDSQeQotlnusePpPGeD12NOPJ-q',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_042fd5e199e2413b901031aefb0328bd.get_object(Bucket='sentimentanalysis-donotdelete-pr-fsgnxpr41f1cmy',Key='twitter.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_excel(body)
data.head()


,coordinates,created_at,hashtags,place,retweet_count,text,user_location,user_name,user_screen_name
0,"-87.73252393,41.79276377",Sat Mar 21 06:11:10 +0000 2020,NaN,"Chicago, IL",0,"Corona Krazy (at @7Eleven in Chicago, IL) http...",773-Chi-CaGo,eL 7 $iEttE,FrEsHCkXO
1,"3.97554,6.58375",Sat Mar 21 06:58:43 +0000 2020,nairamarley covid_19 oja viralvideos tundeednu...,"Epe, Nigeria",1,Junky looking man seen dancing to #nairamarley...,"lagos, Nigeria",Afro Base NG ðŸ‡³ðŸ‡¬,AfroBaseNg
2,"-73.8309,40.7",Sat Mar 21 05:01:38 +0000 2020,covid duxduce corona beer virus,"Queens, NY",0,#covid-19 #duxduce #corona #beer vs #virus @ ...,BKLYN/QUEENZ,DUX DUCE,DUXDUCE
3,"75.4581907,20.7995686",Sat Mar 21 06:33:13 +0000 2020,NaN,"Maharashtra, India",0,Corona @ Samner https://t.co/U2PRr910by,Samaner Tal. Pachora (jalgoan),Amol Patil,bestAmolPatil
4,"-93.2645,44.9774",Sat Mar 21 05:39:51 +0000 2020,minneapolisreggaelegends spring bunny reggae m...,"Minneapolis, MN",1,Stay safe and healthy! #minneapolisreggaelegen...,Minneapolis,Dred I Dread,dredidread


# Preprocessing Function

In [3]:
def preprocessing1(i):
    temp = ''

   
    for i_ in i: 
        if (i_).lower() in "abcdefghijklmnopqrstuvwxyz0123456789 ":
            temp += i_
    i=temp

    i = i.split()
 
    for x in i:
        if 'https' in x:
            i.remove(x)
    i = (" ".join(i))
    #print(i)
    
    return(i)


# Pre-processing Data

In [4]:
data['text'] = data['text'].apply((lambda x: preprocessing1(x)))

In [5]:
data['text'].head()

0                Corona Krazy at 7Eleven in Chicago IL
1    Junky looking man seen dancing to nairamarley ...
2    covid19 duxduce corona beer vs virus Queens Ne...
3                                        Corona Samner
4    Stay safe and healthy minneapolisreggaelegends...
Name: text, dtype: object

# Creating labels

In [6]:
!pip install vaderSentiment

In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [8]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return(score)

In [9]:
def sentiment_score(x):
    
    score = sentiment_analyzer_scores(x)
    if ('fight against' in x) or ('stay alone' in x) or ('break the chain' in x) or ('block the chain' in x) or ('avoid' in x) or ('winning war' in x) or ('fighting war' in x):
        score['compound'] += 0.75

    elif ('tested positive' in x):
        score['compound'] -= 0.75

    negative = score['neg']
    neutral = score['neu']
    positive = score['pos']
    compound = score['compound']

    if compound>0.5:
        new_class = [1]
    elif compound<0:
        new_class = [-1]
    else:
        new_class = [0]
        
    return(new_class)

 

In [10]:
data['New Class'] = data['text'].apply((lambda x: sentiment_score(x)))

In [11]:
data.head()

,coordinates,created_at,hashtags,place,retweet_count,text,user_location,user_name,user_screen_name,New Class
0,"-87.73252393,41.79276377",Sat Mar 21 06:11:10 +0000 2020,NaN,"Chicago, IL",0,Corona Krazy at 7Eleven in Chicago IL,773-Chi-CaGo,eL 7 $iEttE,FrEsHCkXO,[0]
1,"3.97554,6.58375",Sat Mar 21 06:58:43 +0000 2020,nairamarley covid_19 oja viralvideos tundeednu...,"Epe, Nigeria",1,Junky looking man seen dancing to nairamarley ...,"lagos, Nigeria",Afro Base NG ðŸ‡³ðŸ‡¬,AfroBaseNg,[0]
2,"-73.8309,40.7",Sat Mar 21 05:01:38 +0000 2020,covid duxduce corona beer virus,"Queens, NY",0,covid19 duxduce corona beer vs virus Queens Ne...,BKLYN/QUEENZ,DUX DUCE,DUXDUCE,[0]
3,"75.4581907,20.7995686",Sat Mar 21 06:33:13 +0000 2020,NaN,"Maharashtra, India",0,Corona Samner,Samaner Tal. Pachora (jalgoan),Amol Patil,bestAmolPatil,[0]
4,"-93.2645,44.9774",Sat Mar 21 05:39:51 +0000 2020,minneapolisreggaelegends spring bunny reggae m...,"Minneapolis, MN",1,Stay safe and healthy minneapolisreggaelegends...,Minneapolis,Dred I Dread,dredidread,[1]


In [12]:
from IPython.display import FileLink, FileLinks

data.to_csv('Sentiment Analysis.csv', index=False)
data.to_excel('Sentiment Analysis.xlsx', index=False)

FileLinks('.')

./
  Sentiment Analysis.xlsx
  Sentiment Analysis.csv
  data.csv
  data.xlsx